Importing Libraries

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())
# Variables
round_number = 26

from stats import form_data, match_stats, model_libs, form_model, predict_matches

INITIALIZED...


Initially use features just in database, ran PCA and tried to enhance the features that had the most influence...

In [2]:
# Reading in a CSV adds the first index column
training_data = pd.read_csv('round' + str(round_number) + '.csv')
training_data = training_data.drop(training_data.columns[[0]], axis=1)

target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']

td = model_libs._clone_and_drop(training_data, ignore_cols)
(y, X) = model_libs._extract_target(td, target_col)
print('Data Loaded...')

Data Loaded...


In [3]:
models = ['svc', 'gmm', 'kmeans', 'gnb']

def train_models(round_num, X, y):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        if i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
        elif i == 'kmeans':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
        
    return svc, gmm, kmeans, gnb
        
        
def load_models(round_num):
    
    for i in models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'svc':
             svc= joblib.load(model_round)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
        elif i == 'kmeans':
            kmeans = joblib.load(model_round)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
        
        print("Success :: Loaded - " + str(i))
        
    return svc, gmm, kmeans, gnb


# svc, gmm, kmeans, gnb = train_models(round_number, X, y)
# svc, gmm, kmeans, gnb = load_models(round_number)

Find all matches

In [ ]:
upcoming_matches, match_details = predict_matches.get_upcoming_matches()
print(upcoming_matches)

In [ ]:
for i in models:
    if i == 'svc':
        svc_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, svc)
        print(svc_preds)
    elif i == 'gmm':
        gmm_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gmm)
        print(gmm_preds)
    elif i == 'kmeans':
        kmeans_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, kmeans)
        print(kmeans_preds)
    elif i == 'gnb':
        gnb_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gnb)
        print(gnb_preds)

Adds list of predictions to the upcoming matches and puts them in a CSV

In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]
# Add predictions to the end of that DF
results = pd.DataFrame({'SVC': svc_preds, 'GMM': gmm_preds, 'K-Means': kmeans_preds, 'GNB': gnb_preds})
upcoming_matches = upcoming_matches.join(results)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'SVC', 'K-Means', 'GMM', 'GNB']
reordered_matches = reordered_matches[columns]
# upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
reordered_matches.to_csv('predictions_' + str(round_number) + '.csv')

Tune Models
Doesn't really do anything now, need to ask some questions to get better.  Maybe removing some features will help.

In [ ]:
for i in models:
    form_model.build_tuned_model(X, y, i)

Trying to remove some useless features

In [ ]:
print(training_data.shape)
reduced_data = form_data.variance_features(training_data)
print(reduced_data.shape)

In [ ]:
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
              scoring='accuracy')
rfecv.fit(X, y)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

# Need to normalize the data
"""norm_data = X.copy(deep=True)
norm_data = norm_data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
X_new = SelectKBest(chi2, k=50).fit_transform(norm_data, y)
print(X.shape)"""

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE

# Create the RFE object and rank each pixel
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)
print(rfe.ranking_)


In [ ]:
ranked_features = pd.DataFrame(index=range(448))
for a in range(len(rfe.ranking_)):
    for i in range(len(rfe.ranking_)):
        if rfe.ranking_[i] == a+1:
            ranked_features = ranked_features.join(X.ix[:,i])
            
display(ranked_features.describe())

Let's try to reduce the number of features to at least 25 although optimal seems likely around ~8 features

In [ ]:
for x in xrange(5, 25):
    print('X :: {}'.format(x))
    svc, gmm, kmeans, gnb = train_models(round_number, ranked_features.iloc[:, 1:x], y)
    print(' ============================ ')
    

try PCA on original training data 

In [ ]:
pca = PCA(n_components=5)
pca.fit(X)
# Generate PCA results plot
pca_results = rs.pca_results(X, pca)

In [ ]:
# TODO: Apply a PCA transformation the good data
reduced_data = pca.transform(X)
# display(pd.DataFrame(np.round(reduced_data, 4), columns = ['D1', 'D2', 'D3', 'D4', 'D5']))
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

In [ ]:
Try PCA on ranked features 

In [ ]:
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

In [ ]:
pca = PCA(n_components=5)
pca.fit(ranked_features.iloc[:, 1:23]) # 23 was just the highest value when testing
# Generate PCA results plot
pca_results = rs.pca_results(ranked_features.iloc[:, 1:23], pca)

In [ ]:
# TODO: Apply a PCA transformation the good data
reduced_data = pca.transform(ranked_features.iloc[:, 1:23])
# display(pd.DataFrame(np.round(reduced_data, 4), columns = ['D1', 'D2', 'D3', 'D4', 'D5']))
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

Let's try removing ties similar to the other project

In [4]:
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']

filtered_data = training_data[training_data.points != 1]
fd = model_libs._clone_and_drop(filtered_data, ignore_cols)
(f_y, f_X) = model_libs._extract_target(fd, target_col)
print(f_X.shape)

(298, 77)


In [5]:
svc, gmm, kmeans, gnb = train_models(round_number, f_X, f_y)

Training SVC Modeling
F1 score for training set: 0.858333333333
F1 score for test set: 0.566666666667
Finished SVC Modeling
Training GMM Modeling
F1 score for training set: 0.163265306122
F1 score for test set: 0.190476190476
('Scores :', [0.097067449979951009])
Finished GMM Modeling
Training K-Means Modeling
F1 score for training set: 0.232876712329
F1 score for test set: 0.241379310345
Finished SVC Modeling
Finished K-Means Modeling
Training Gaussian NB Modeling
F1 score for training set: 0.769874476987
F1 score for test set: 0.714285714286
Finished Gaussian NB Modeling


//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
//anaconda/lib/python2.7/site-packages/sklearn/